In [1]:
import numpy as np
import sklearn_crfsuite
import pandas as pd
import nltk
import pickle
from rdflib import Graph, URIRef
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import csv
import numpy as np
import os
import rdflib
import pandas as pd
from sklearn.metrics import pairwise_distances
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
import json
import time
import atexit
import getpass
import requests  # install the package via "pip install requests"
from collections import defaultdict
import pandas as pd
from rdflib import Graph
import rdflib
import numpy as np
import csv
from sklearn.metrics import pairwise_distances
from statistics import mode

import time
import atexit
import getpass
import requests  # install the package via "pip install requests"
from collections import defaultdict


#from sparknlp.annotator import *
#from sparknlp.base import *
#import sparknlp
#import spark
import urllib.request

#from sparknlp.annotator import *
#from sparknlp.base import *
#import sparknlp
#spark = sparknlp.start()

#from sparknlp.pretrained import PretrainedPipeline

###
#import spacy

## Answer factual questions
Model training need to be applied only once. The function is at the bottom

In [2]:
class NerTrainer:
    #Most of the funcitons are from the Graphical Models Tutorial from the class

    def __init__(self):
        self.x_train = pd.read_csv('engtrain_clean.csv', encoding="ISO-8859-1")
        self.x_test = pd.read_csv('engtest_clean.csv', encoding="ISO-8859-1")
        print("Loaded data & initialized NER-model")

    def collate(self, dataframe):
        agg_func = lambda s: [(w, pos, t) for w, pos, t in
                              zip(s['WORD'].values.tolist(), s['POS'].values.tolist(), s['ï»¿TAG'].values.tolist())]
        grouped = dataframe.groupby('SENTENCE').apply(agg_func)
        return list(grouped)

    # counts sentences in given input data
    def count_sentences(self, data):
        counter = 0
        for index, row in data.iterrows():
            data.at[index, 'SENTENCE'] = counter
            if pd.isna(data.iloc[index, 0]):
                counter += 1
        data["POS"] = "0"
        data = data.dropna()
        data = data.reset_index(drop=True)
        return data

    def word2features(self, sent, i):
        word = sent[i][0]
        postag = sent[i][1]

        features = {
            'word.lower()': word.lower(),  # the word in lowercase
            'word[-3:]': word[-3:],  # last three characters
            'word[-2:]': word[-2:],  # last two characters
            'word.isupper()': word.isupper(),  # true, if the word is in uppercase
            'word.istitle()': word.istitle(),
            # true, if the first character is in uppercase and remaining characters are in lowercase
            'word.isdigit()': word.isdigit(),  # true, if all characters are digits
            'postag': postag,  # POS tag
            'postag[:2]': postag[:2],  # IOB prefix
        }

        if i > 0:
            word1 = sent[i - 1][0]  # the previous word
            postag1 = sent[i - 1][1]  # POS tag of the previous word
            features.update({
                '-1:word.lower()': word1.lower(),
                '-1:word.istitle()': word1.istitle(),
                '-1:word.isupper()': word1.isupper(),
                '-1:postag': postag1,
                '-1:postag[:2]': postag1[:2],
            })  # add some features of the previous word
        else:
            features['BOS'] = True  # BOS: begining of the sentence

        if i < len(sent) - 1:
            word1 = sent[i + 1][0]  # the next word
            postag1 = sent[i + 1][1]  # POS tag of the next word
            features.update({
                '+1:word.lower()': word1.lower(),
                '+1:word.istitle()': word1.istitle(),
                '+1:word.isupper()': word1.isupper(),
                '+1:postag': postag1,
                '+1:postag[:2]': postag1[:2],
            })  # add some features of the next word
        else:
            features['EOS'] = True  # EOS: end of the sentence
        return features

    def sent2features(self, sent):
        return [self.word2features(sent, i) for i in range(len(sent))]

    def sent2labels(self, sent):
        return [label for _, _, label in sent]

    def tag_data(self, data_to_tag):
        tags = []
        x_train = data_to_tag.groupby(['SENTENCE'])
        x_train = x_train.apply(np.mean)

        # x_train.shape[0] will give us the number of sentences to loop through and tag them using nltk lib
        # first we locate the sentence in our input data, then we hand the sentence as a list of words to the nltk function which tags the words.
        for i in range(x_train.shape[0]):
            sentence = data_to_tag.loc[data_to_tag["SENTENCE"] == i]
            sentence = sentence['WORD'].tolist()
            tag = nltk.pos_tag(sentence)
            tags.append(tag)

        #now we add the found tags to the actual data in the data set
        counter = 0
        for sentence in tags:
            for word in sentence:
                data_to_tag.at[counter, 'POS'] = word[1]
                counter += 1
        print("data to tag in ""tag_data()"":" )
        print(data_to_tag)
        print("#######################################################")
        return data_to_tag

    #code copied from graphical_model tutorial and collected under one train function
    def train(self):
        self.x_train = self.count_sentences(self.x_train)
        self.x_test = self.count_sentences(self.x_test)

        self.x_train = self.tag_data(self.x_train)
        self.x_test = self.tag_data((self.x_test))

        self.x_train = self.collate(self.x_train)
        self.x_test = self.collate((self.x_test))

        X_train, y_train = [self.sent2features(s) for s in self.x_train], [self.sent2labels(s) for s in self.x_train]
        X_test, y_test = [self.sent2features(s) for s in self.x_test], [self.sent2labels(s) for s in self.x_test]

        crf = sklearn_crfsuite.CRF(
            algorithm='l2sgd',  # l2sgd: Stochastic Gradient Descent with L2 regularization term
            max_iterations=1000,  # maximum number of iterations
        )
        crf_model = crf.fit(X_train, y_train)
        return  crf_model

    #predict tags of given input sentence
    def predict_tags(self, model, sentence):
        text = []

        #create list of workds from input sentence
        words = word_tokenize(sentence)
        tagged_words = nltk.pos_tag(words)
        text.append(tagged_words)

        empty_list = word_tokenize("")
        empty_tagged_words = nltk.pos_tag(empty_list)
        text.append(empty_tagged_words)

        X_train = [self.sent2features(s) for s in text]
        y = model.predict(X_train)

        #remove any punctuation mark
        if sentence[0][-1][0] in ['?', '!', '¨.']:
            y[0][-1] = '0'

        #Create output dataframe with two new columns and the predicted tag
        tagged_words = np.asarray(tagged_words)
        df_res = pd.DataFrame(tagged_words, columns=['WORD', 'POS'])
        df_res['TAG'] = y[0]

        return df_res

In [30]:
# Class to identify the input question. Question content and type.
class QuestionAnalyser:
    def __init__(self):
        print("Initialized Question Analyser")
        self.exceptional_predicates = {'MPA film rating':"P1657", 'Box Office Mojo film ID':"P1237", 'duration':"P2047", 'FilmAffinity ID':"P480", "cast member":"P161",
                                       'Kinopoisk film ID':"P2603", "producer":'P162', 'exploitation visa number':"P2755", 'average shot length':"P2208", 'film script':"P3816", 'budget':"P2769"}

    #Funciton to identify if question is about the director ot the actor.
    #Counts nouns and verbs that are related to either. Role with highest score is designated as question content.
    def identity_counter(self, lemma_verbs, lemma_nouns):

        lemma_words = lemma_verbs + lemma_nouns

        candidates = {"director" : ['director', "filmmaker",'direct', 'produce', 'filmed', 'lead'],
                      "actor" : ['actor', 'role', 'star', 'act', 'play', 'star'],
                      "screenwriter":  ['screenwriter', 'scriptwriter', 'author', 'screenwrite', 'write', 'script'],
                      "MPA film rating" : ['mpaa', 'rating', 'ranking', 'grade', 'rate', 'rank', 'grade'],
                      "image" : ['image', 'picture', 'face', 'look', "show"],
                      "genre" : ['genre', 'type', 'style', 'category'],
                      "box office" : ["box", "office", "ticket", "booth"],
                      "publication date": ["publication", "publish", "release"],
                      "cast member": ["cast", "casting"],
                      "producer": ["producer", 'produced', "produce", "maker", "make", "production"],
                      "executive producer": ["producer",'produced', "produce", "executive", "maker", "make"],
                      "based on": ["base", "based"],
                      "country of origin" : ["origin"],
                      "filming location": ["location", "place", "locate", "film", "shoot"],
                      "narrative location": ["location", "place", "locate", "narrate", "narrative"],
                      "director of photography": ["photography"],
                      "Box Office Mojo film ID": ["box", "office", "ticket", "booth", "Mojo", "ID", "id"],
                      "set in period": ["period"],
                      "duration": ["duration"],
                      "film editor": ["editor", "edit", "compiler", "compile"],
                      "FilmAffinity ID": ["FilmAffinity", "filmaffinity"],
                      "Filmportal": ["Filmportal", "filmportal"],
                      "main subject": ["subject", "topic"],
                      "composer": ["composer", "compose"],
                      "distributed by": ["distributer", "distribute", "trader", "trade", "seller"],
                      "costume designer": ["costume", "outfit", "wardrobe", "dress"],
                      "production designer": ["producer",'produced', "produce", "executive", "maker", "make", "designer", "design"],
                      "Kinopoisk film ID": ["Kinopoisk", "kinopoisk"],
                      "average shot length": ["average", "shot", "length"],
                      "exploitation visa number": ["visa", "exploitation"],
                      "original film format": ["original", "film", "format"],
                      "film script": ["script", "scripte"],
                      "title": ["title"],
                      "award received": ["award"],
                      "media franchise": ["franchise"],
                      "part of the series": ["series"],
                      "budget": ["budget", "cost"],

                      ############################

                      }


        counter = {"director": 0, "actor": 0, "screenwriter": 0, "MPA film rating": 0, "image": 0, "genre": 0,
                   "box office": 0, "publication date": 0, "cast member": 0, "producer": 0, "executive producer":0,
                   "based on": 0, "country of origin": 0, "filming location": 0, "narrative location": 0, "director of photography": 0,
                   "Box Office Mojo film ID":0, "set in period": 0, "duration": 0, "film editor": 0, "FilmAffinity ID": 0,
                   "Filmportal ID": 0, "main subject": 0, "composer": 0, "distributed by": 0,  "costume designer": 0,
                   "production designer": 0, "Kinopoisk film ID": 0, "exploitation visa number": 0, "average shot length": 0,
                   "original film format": 0, "film script": 0, "title": 0, "award": 0, "media franchise": 0, "part of the series": 0,
                   "budget": 0
                   }

        #check occurences of words in input sentence
        for key in candidates.keys():
            for w in candidates[key]:
                if w in lemma_words:
                    counter[key] += 1

        max_value = max(counter, key=counter.get)

        #double check for box office and avg shot length since it should appear in word combinations
        if max_value == "Box Office Mojo film ID":
            lemma_lower = [w.lower() for w in lemma_words]
            if "id" not in lemma_lower:
                max_value = "box office"

        if max_value == "average shot length":
            counter["average shot length"] -= 1
            max_value = max(counter, key=counter.get)
        if max_value == "original film format":
            counter["original film format"] -= 1
            max_value = max(counter, key=counter.get)


        #decide between location properties:
        if max_value in ["country of origin", "filming location", "narrative location"]:
            if "origin" in lemma_verbs:
                max_value = "country of origin"
            elif "film" or "shoot" in lemma_verbs:
                max_value = "filming location"
            else:
                max_value = "narrative location"

        #decide between the production rolees:
        if max_value in ["production designer", "executive producer"]:
            if "designer" in lemma_words:
                max_value = "production designer"
            elif  "design" in lemma_words:
                max_value = "production designer"
            elif "executive" in lemma_words:
                max_value = "executive producer"
            else: max_value = "producer"

        print("max valu in identity counter: ")
        print(max_value, counter[max_value])


        if counter[max_value] == 0:
            return False
        else:
            return max_value

    def identify_predicate(self, question, recurrence = False):

        # Get untaggde words
        untagged_words = question[question["TAG"]=='0']
        verbs = untagged_words[untagged_words['POS'].isin(['VP', 'VBD', 'VBG', 'VBP', 'VBZ'])]['WORD']
        nouns = untagged_words[untagged_words['POS'].isin(['NN', 'NNP', 'NNPS', "JJ"])]['WORD']

        #Lemmatize means to get the root of the word
        lemma = WordNetLemmatizer()
        lemma_verbs = [lemma.lemmatize(v) for v in verbs]
        lemma_nouns = [lemma.lemmatize(v) for v in nouns]
        print(lemma_nouns)
        print(lemma_verbs)

        ########################################################
        pred = self.identity_counter(lemma_nouns = lemma_nouns, lemma_verbs=lemma_verbs)

        #Sometimes we find that the predicate is bound to the tile by tags. Is we cant find a pred,
        #We remove the tag from every word that occurs before a "of" word that was tagged.
        if recurrence == False:
            if pred == False:
                looks = ["look", "looks"]
                if any(c in question["WORD"].unique() for c in looks):
                    try:
                        idx = question.index[question['WORD'] == "look"].tolist()
                        question.iloc[idx[0]:]["TAG"] = '0'
                        print("changed look like tags")
                        print(question)
                        pred = self.identify_predicate(question, recurrence=True)

                    except:
                        try:
                            idx = question.index[question['WORD'] == "looks"].tolist()
                            question.iloc[idx[0]:]["TAG"] = '0'
                            print("changed look like tags")
                            print(question)
                            pred = self.identify_predicate(question, recurrence=True)
                        except:
                            pred = False

                else:
                    try:
                        idx = question.index[question['WORD'] == "of"].tolist()
                        question.iloc[:idx[0]+1]["TAG"] = '0'
                        print("changed tags")
                        print(question)
                        pred = self.identify_predicate(question, recurrence=True)
                    except:
                        pred = False
        # check if start of usbject could be identified. If not, set in manually-
        if pred == False:
            print("changing tags")
            if "B-TITLE" not in question["TAG"].unique():
                print("changning Tags because no title tags found")
                try:
                    idx = question.index[question["TAG"]=="I-PLOT"].tolist()
                    question.iloc[idx[0]:idx[0]+1]["TAG"] = 'B-TITLE'
                    question.loc[question['TAG'] == "I-PLOT", 'TAG'] = 'I-TITLE'
                    pred = self.identify_predicate(question, recurrence = True)

                except:
                    pass

        print("pred to return:")
        print(pred)
        return pred

    def handle_exceptional_predicates(self, predicate):
        print("excepional predicate triggered")
        return self.exceptional_predicates[predicate]

    def predicate_corrector(self, question):
        looks = ["look", "looks"]
        if any(c in question["WORD"].unique() for c in looks):
            try:
                idx = question.index[question['WORD'] == "look"].tolist()
                question.iloc[idx[0]:]["TAG"] = '0'
                print("changed look like tags")
                print(question)
                pred = self.identify_predicate(question, recurrence=True)

            except:
                try:
                    idx = question.index[question['WORD'] == "looks"].tolist()
                    question.iloc[idx[0]:]["TAG"] = '0'
                    print("changed look like tags")
                    print(question)
                    pred = self.identify_predicate(question, recurrence=True)
                except:
                    pred = False

        else:
            try:
                idx = question.index[question['WORD'] == "of"].tolist()
                question.iloc[:idx[0]+1]["TAG"] = '0'
                print("changed tags")
                print(question)
                pred = self.identify_predicate(question, recurrence=True)
            except:
                pred = False
        return pred

In [4]:
r = "BaaaaaX "

if r[-1].isspace():
    print(r[:-1])

BaaaaaX


In [5]:
class TypoCorrector:

    @staticmethod
    def remove_last_digit_space(input):
        # replace minus with dash
        if input[-1].isspace():
            input = input[:-1]
        return input

    @staticmethod
    def change_minus_sign(input):
        # replace minus with dash
        if "-" in input:
            input = input.replace("-", "–")

        return input

    @staticmethod
    def change_dash_sign(input):
        # replace minus with dash
        if "-" in input:
            input = input.replace("–","-")

        return input

    @staticmethod
    def title_corrector(film, actor):

        # To perform the query, we must remove punctuations.
        if len(film) >= 1:
            if film[-1] in ['?', '!', '.']:
                film = film[:-1]
        # intermediate punctions have a space in front of them that needs to be removed
        if " :" in film:
            film = film.replace(" :", ":")

        if len(actor) >= 1:
            if actor[-1] in ['?', '!', '.']:
                actor = actor[:-1]

        return film, actor

In [6]:
class  ModelTrainer:

    def __init__(self, ner_model, training_data, test_data):
        self.ner_model = ner_model
        self.x_train = pd.read_csv(training_data, encoding="ISO-8859-1")
        self.x_test = pd.read_csv(test_data, encoding="ISO-8859-1")
        print("Loaded data successfully")



    def train_model(self):
        self.ner_model.train(self.x_train,self.x_test)

In [65]:
class Crowdsourcing:

    def __init__(self):
        self.df = pd.read_csv('D:/deanh/Uni/Master/HS22/Advanced AI/Project/crowdsourcing/crowd_data.tsv', sep='\t')
        self.df = self.clean_crowdsourcingfile()
        self.kappas = self.calculate_kappas(self.df)


    def clean_crowdsourcingfile(self):
        #remove % sign from approvalrate to handle as integer
        self.df["LifetimeApprovalRate"] = [int(word[:-1]) for word in self.df['LifetimeApprovalRate']]

        # We define upper and lower bounds for response time and approval rate by using mean and standard deviation
        # In believe that workers outside of these bounds are malicious. We add +5% to all bounds to include valid edge cases
        # We also drop rows that indicate as CORRECT but say the know an error and where to handle it

        x = "WorkTimeInSeconds"
        y = "LifetimeApprovalRate"

        # mean_approvalRate = self.df[y].mean()
        # std_approvalRate = self.df[y].std()
        #
        # mean_responseTime = self.df[x].mean()
        # std_responseTime = self.df[x].std()
        #
        # upperbound_responseTime = mean_responseTime + std_responseTime + 5
        # lowerbound_responseTime = mean_responseTime - std_responseTime - 5
        #
        # upperbound_approvalRate = mean_approvalRate + std_approvalRate + 5
        # lowerbound_approvalRate = mean_approvalRate - std_approvalRate -5
        #
        # self.df = self.df.drop(self.df[(self.df.WorkTimeInSeconds > upperbound_responseTime)].index)
        # self.df = self.df.drop(self.df[(self.df.WorkTimeInSeconds < lowerbound_responseTime)].index)
        #
        # self.df = self.df.drop(self.df[(self.df.LifetimeApprovalRate > upperbound_approvalRate)].index)
        # self.df = self.df.drop(self.df[(self.df.LifetimeApprovalRate < lowerbound_approvalRate)].index)

        ###################################################################################################################
        workers = pd.DataFrame(self.df["WorkerId"])
        malicous_workers_Id = []

        for e in workers["WorkerId"]:
            df_n = pd.DataFrame(self.df.loc[self.df["WorkerId"] == e, "AnswerID"])
            if len(df_n["AnswerID"].unique()) != 2:
                malicous_workers_Id.append(e)

        for e in malicous_workers_Id:
            self.df = self.df.drop(self.df[(self.df.WorkerId == e)].index)

        self.df = self.df.drop(self.df[(self.df.WorkTimeInSeconds <= 5)].index)
        self.df = self.df.drop(self.df[(self.df.LifetimeApprovalRate <= 40)].index)


        #There is a type in Input2ID column
        self.df["Input2ID"] = self.df["Input2ID"].replace("wdt:.P344", "wdt:P344")

        return self.df

    def crowdsourcing_answer(self, sub, pred):
        rows = self.df.loc[self.df["Input1ID"] == "wd:"+sub]
        rows = rows.loc[rows["Input2ID"] == "wdt:"+pred]

        agree = rows.loc[rows["AnswerLabel"] == "CORRECT"]
        disagree = rows.loc[rows["AnswerLabel"] == "INCORRECT"]
        value = rows.iloc[0]['Input3ID']
        batch_id = rows.iloc[0]["HITTypeId"]
        kappa = self.kappas[batch_id]

        answer = {"batch": batch_id, "kappa": kappa, "agreement": agree.shape[0], "disagreement":disagree.shape[0], "value" : str(value), "answer": "tie"}
        if agree.shape[0] > disagree.shape[0]:
            answer["verdict"] = "True"
        if agree.shape[0] < disagree.shape[0]:
            answer["verdict"] = "False"
            for e in disagree["FixValue"].unique():
                if isinstance(e, str):
                    if e != "I don't understand":
                        answer["value"] = e
                        answer["verdict"] = "corrected"
        if agree.shape[0] == disagree.shape[0]:
            answer["verdict"] = "tie"

        return answer

    def calculate_kappas(self, df):

        batch_names = df["HITTypeId"].unique()
        voting_list = {}
        for batch_name in batch_names:
            batch = df.loc[df["HITTypeId"] == batch_name]
            voters = batch["WorkerId"].unique()
            voting_collection = [[] for v in range(len(voters))]

            for i in range (len(voters)):
                worker = voters[i]
                votes = batch.loc[batch["WorkerId"] == worker]["AnswerLabel"].tolist()
                voting_collection[i] = (votes)
            voting_list[batch_name] = voting_collection

        kappas = {}
        for key in voting_list.keys():
            vote_values = voting_list[key]
        #pe
            correct = 0
            incorrect = 0
            for e in vote_values:
                for i in e:
                    if i == "CORRECT":
                        correct += 1
                    elif i == "INCORRECT":
                        incorrect += 1
            sum = correct + incorrect
            pe = (correct/sum)**2 + (incorrect/sum)**2

            #po
            N = len(vote_values[0])
            n = len(vote_values)
            first_part = 1 / (N*n*(n-1))

            sec_part = 0
            for i in range(len(vote_values[0])):
                correct_counter = 0
                incorrect_counter  = 0
                for e in range(len(vote_values)):
                    if vote_values[e][i] == "CORRECT":
                        correct_counter += 1
                    elif vote_values[e][i] == "INCORRECT":
                        incorrect_counter += 1

                sec_part += correct_counter**2
                sec_part += incorrect_counter**2

            po =  first_part * (sec_part - N * n)

            k = (po - pe) / (1-pe)
            kappas[key] = round(k, 2)

        return kappas

In [8]:
class KGLoader:
    def __init__(self):
        pass

    def load_KG(self):
        print("Loading knowledge graph from source")
        movie_graph = Graph()
        movie_graph.parse(source='datasets/14_graph.nt', format='turtle')
        print("succesfully loaded knowledge graph")
        return movie_graph

    def load_embeddings(self, graph):
        #add path parameter to retrive dictionaries?

        DDIS = rdflib.Namespace('http://ddis.ch/atai/')
        RDFS = rdflib.namespace.RDFS
        SCHEMA = rdflib.Namespace('http://schema.org/')

        # load the embeddings
        entity_emb = np.load(os.path.join('D:\deanh','Uni','Master','HS22','Advanced AI','Project', 'embedding_data', 'entity_embeds.npy'))
        relation_emb = np.load(os.path.join('D:\deanh','Uni','Master','HS22','Advanced AI','Project', 'embedding_data', 'relation_embeds.npy'))

        # load the dictionaries
        with open(os.path.join('D:\deanh','Uni','Master','HS22','Advanced AI','Project', 'embedding_data', 'entity_ids.del'), 'r') as ifile:
            ent2id = {rdflib.term.URIRef(ent): int(idx) for idx, ent in csv.reader(ifile, delimiter='\t')}
            id2ent = {v: k for k, v in ent2id.items()}
        with open(os.path.join('D:\deanh','Uni','Master','HS22','Advanced AI','Project', 'embedding_data', 'relation_ids.del'), 'r') as ifile:
            rel2id = {rdflib.term.URIRef(rel): int(idx) for idx, rel in csv.reader(ifile, delimiter='\t')}
            id2rel = {v: k for k, v in rel2id.items()}

        ent2lbl = {ent: str(lbl) for ent, lbl in graph.subject_objects(RDFS.label)}
        lbl2ent = {lbl: ent for ent, lbl in ent2lbl.items()}

        embedding = (entity_emb, relation_emb, ent2id, id2ent, rel2id, id2rel, ent2lbl, lbl2ent)

        return embedding

In [9]:
# Returns a SQL statement as a string
class QueryProducer:
    WDT = rdflib.Namespace('http://www.wikidata.org/prop/direct/')
    WD = rdflib.Namespace("http://www.wikidata.org/entity/")

    def __init__(self, knowledgegraph):
        self.moviegraph = knowledgegraph

    def director_query(self, title):
        query ='''
            SELECT ?name
            WHERE {
            ?film ns1:P31 <http://www.wikidata.org/entity/Q11424> ;
                  rdfs:label "''' + title + '''"@en ;
                  ns1:P57 ?director .
            ?director rdfs:label ?name
            }
            '''
        return query

    # returns a dict, with all film titles and their entity Id
    def all_films_id_query(self):
        WDT = rdflib.Namespace('http://www.wikidata.org/prop/direct/')
        WD = rdflib.Namespace("http://www.wikidata.org/entity/")

        query ='''
        PREFIX wd: <http://www.wikidata.org/entity/>
        PREFIX wds: <http://www.wikidata.org/entity/statement/>
        PREFIX wdv: <http://www.wikidata.org/value/>
        PREFIX wdt: <http://www.wikidata.org/prop/direct/>
        PREFIX wikibase: <http://wikiba.se/ontology#>
        PREFIX p: <http://www.wikidata.org/prop/>
        PREFIX ps: <http://www.wikidata.org/prop/statement/>
        PREFIX pq: <http://www.wikidata.org/prop/qualifier/>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX bd: <http://www.bigdata.com/rdf#>

        SELECT ?item ?label
            WHERE{
            ?item wdt:P31/wdt:P279* <http://www.wikidata.org/entity/Q11424> .
            ?item rdfs:label ?label .
            FILTER(LANG(?label) = "en").
        }
                    '''
        response = moviegraph.query(query)
        id_to_film = {e[len(WD):]: str(i) for e,i in response}
        film_to_id = {value:key for key, value in id_to_film.items()}

        # set all movie titles to lowercase
        film_to_id = {k.lower(): v for k, v in film_to_id.items()}
        print("found normal films; " + "dict has size " + str(len(film_to_id)))

        sub_categories = self.subcategories_films_query()
        print("found subcategories; " + "dict has size " + str(len(sub_categories)))
        sub_sub_categories = self.subcategories_of_subcategories_query()
        print("found sub_subcategories; " + "dict has size " + str(len(sub_sub_categories)))
        animes = self.all_animes_id_query()
        print(print("found animes; " + "dict has size " + str(len(animes))))

        film_to_id = {**film_to_id, **sub_categories, **sub_sub_categories, **animes}
        print("the final film dict has size " + str(len(film_to_id)))


        return  film_to_id

        # returns a dict, with all anime titles and their entity Id

    def subcategories_films_query(self):
        WDT = rdflib.Namespace('http://www.wikidata.org/prop/direct/')
        WD = rdflib.Namespace("http://www.wikidata.org/entity/")
        # we iterate through the subcategories of the film category and get all subclasses
        # then we iterate grough these subclasses and collect all movies
        subcategory_query = '''
            SELECT ?subcategory
            WHERE {
            ?subcategory ns1:P279 <http://www.wikidata.org/entity/Q11424> ;
            }
            '''
        sub_response = self.moviegraph.query(subcategory_query)
        subcategories = [str(_.subcategory) for _ in sub_response]

        id_to_film = {}
        for category in subcategories:
            query = '''
            SELECT ?item ?title
            WHERE {
            ?item ns1:P31 <''' + category + '''> .
                  ?item rdfs:label ?title .
            }
            '''

            response = self.moviegraph.query(query)
            for e in response:
                id_to_film[str(e.item)[len(WD):]] = str(e.title)

        film_to_id = {value:key for key, value in id_to_film.items()}

         # set all movie titles to lowercase
        film_to_id = {k.lower(): v for k, v in film_to_id.items()}
        return film_to_id

    def subcategories_of_subcategories_query(self):
        # AGAIN we iterate through the subcategories of the film category and get all
        # But we also need to look at all the subcategories of the subcategory
        # then we iterate grough these subclasses and collect all movies
        WDT = rdflib.Namespace('http://www.wikidata.org/prop/direct/')
        WD = rdflib.Namespace("http://www.wikidata.org/entity/")
        subcategory_query = '''
            SELECT ?subcategory
            WHERE {
            ?subcategory ns1:P279 <http://www.wikidata.org/entity/Q11424> ;
            }
            '''
        sub_response = self.moviegraph.query(subcategory_query)
        subcategories = [str(_.subcategory) for _ in sub_response]

        print("len of subcategories: " + str(len(subcategories)))


        for category in subcategories:
            sub_subcategory_query = '''
                SELECT ?sub_subcategory
                WHERE {
                ?sub_subcategory ns1:P279 <''' + category + '''> ;
                }
            '''
        sub_sub_response = self.moviegraph.query(sub_subcategory_query)
        sub_subcategories = [str(_.sub_subcategory) for _ in sub_sub_response]

        id_to_film = {}
        for category in sub_subcategories:
                query = '''
            SELECT ?item ?title
            WHERE {
            ?item ns1:P31 <''' + category + '''> .
                  ?item rdfs:label ?title .
            }
            '''

                response = self.moviegraph.query(query)
                for e in response:
                    id_to_film[str(e.item)[len(WD):]] = str(e.title)

        film_to_id = {value:key for key, value in id_to_film.items()}

        # set all movie titles to lowercase
        film_to_id = {k.lower(): v for k, v in film_to_id.items()}
        return film_to_id

    def all_animes_id_query(self):
        WDT = rdflib.Namespace('http://www.wikidata.org/prop/direct/')
        WD = rdflib.Namespace("http://www.wikidata.org/entity/")

        query ='''
        PREFIX wd: <http://www.wikidata.org/entity/>
        PREFIX wds: <http://www.wikidata.org/entity/statement/>
        PREFIX wdv: <http://www.wikidata.org/value/>
        PREFIX wdt: <http://www.wikidata.org/prop/direct/>
        PREFIX wikibase: <http://wikiba.se/ontology#>
        PREFIX p: <http://www.wikidata.org/prop/>
        PREFIX ps: <http://www.wikidata.org/prop/statement/>
        PREFIX pq: <http://www.wikidata.org/prop/qualifier/>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX bd: <http://www.bigdata.com/rdf#>

        SELECT ?item ?label
            WHERE{
            ?item wdt:P31 <http://www.wikidata.org/entity/Q20650540> .
            ?item rdfs:label ?label .
            FILTER(LANG(?label) = "en").
        }
                    '''
        response = self.moviegraph.query(query)
        id_to_film = {e[len(WD):]: str(i) for e,i in response}
        film_to_id = {value:key for key, value in id_to_film.items()}

        # set all movie titles to lowercase
        film_to_id = {k.lower(): v for k, v in film_to_id.items()}

        return  film_to_id

    def all_actors_id_and_IMDb_query(self):
        WDT = rdflib.Namespace('http://www.wikidata.org/prop/direct/')
        WD = rdflib.Namespace("http://www.wikidata.org/entity/")
        query ='''
            PREFIX wd: <http://www.wikidata.org/entity/>
            PREFIX wds: <http://www.wikidata.org/entity/statement/>
            PREFIX wdv: <http://www.wikidata.org/value/>
            PREFIX wdt: <http://www.wikidata.org/prop/direct/>
            PREFIX wikibase: <http://wikiba.se/ontology#>
            PREFIX p: <http://www.wikidata.org/prop/>
            PREFIX ps: <http://www.wikidata.org/prop/statement/>
            PREFIX pq: <http://www.wikidata.org/prop/qualifier/>
            PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
            PREFIX bd: <http://www.bigdata.com/rdf#>

            SELECT ?item ?label ?IMDb_ID
                WHERE{
                ?item wdt:P31 wd:Q5 .
                ?item wdt:P106 wd:Q33999 .
                ?item rdfs:label ?label .
                ?item wdt:P345 ?IMDb_ID  .
            }


            '''

        response = self.moviegraph.query(query)
        ID_to_actor = {e[len(WD):]: str(i) for e,i, IDMb in response}
        actor_to_Id = {value:key for key, value in ID_to_actor.items()}

        ID_to_actor = {e[len(WD):]: str(IDMb) for e,i, IDMb in response}

        return [actor_to_Id, ID_to_actor]

    def predicateID_query(self, proposition):

        query ="""

        prefix wdt: <http://www.wikidata.org/prop/direct/>
        prefix wd: <http://www.wikidata.org/entity/>

        SELECT ?obj WHERE {
            ?obj rdfs:label """ + '"' + proposition + '"' + """@en .

        }
        """
        entity =self.moviegraph.query(query)
        df = pd.DataFrame(entity.bindings)
        url = str((df.iloc[0][0]))
        id = url[len(QueryProducer.WDT):]

        return id

    def label_query(self, id):
        query ='''
        prefix wdt: <http://www.wikidata.org/prop/direct/>
        prefix wd: <http://www.wikidata.org/entity/>
        SELECT  *
        WHERE {
            '''+id+''' rdfs:label ?label .
            FILTER (langMatches( lang(?label), "EN" ) )
          }

                '''

        response = moviegraph.query(query)
        df = pd.DataFrame(response.bindings)
        label = str((df.iloc[0][0]))
        return label

    def genre_id(self, title):
        query ="""
            prefix wdt: <http://www.wikidata.org/prop/direct/>
            prefix wd: <http://www.wikidata.org/entity/>

            SELECT ?obj WHERE {
                ?item wdt:P31/wdt:P279* wd:Q201658 .
                ?obj rdfs:label """ + '"' + title + '"' + """@en .

            }
            """
        response = self.moviegraph.query(query)
        response = [str(i[0]) for i in response]

        return  response[0][len(QueryProducer.WD):]

    def any_property_value_query(self, title, id, propertyname):
        WDT = rdflib.Namespace('http://www.wikidata.org/prop/direct/')
        WD = rdflib.Namespace("http://www.wikidata.org/entity/")
        
        if " " in propertyname:
            propertyname = propertyname.replace(" ", "_")

        query ='''
                SELECT ?name
                WHERE {
                ?film ns1:P31 <http://www.wikidata.org/entity/Q11424> ;
                      rdfs:label "''' + title + '''"@en ;
                      ns1:'''+id+''' ?'''+propertyname+''' .
                ?'''+propertyname+''' rdfs:label ?name
                }
                '''
        return query

    def any_property_value(self, title, id, propertyname):
        WDT = rdflib.Namespace('http://www.wikidata.org/prop/direct/')
        WD = rdflib.Namespace("http://www.wikidata.org/entity/")

        if " " in propertyname:
            propertyname = propertyname.replace(" ", "_")

        query ='''
                SELECT ?name
                WHERE {
                ?film ns1:P31 <http://www.wikidata.org/entity/Q11424> ;
                      rdfs:label "''' + title + '''"@en ;
                      ns1:'''+id+''' ?'''+ propertyname+''' .
                ?'''+propertyname+''' rdfs:label ?name
                }
                '''
        response = moviegraph.query(query)
        df = pd.DataFrame(response.bindings)
        try:
            value = str((df.iloc[0][0]))
        except:
            value = "nothing"
        return value

    def all_genre_films(self, genre):
        query ='''
        prefix wd: <http://www.wikidata.org/entity/>
        prefix wdt: <http://www.wikidata.org/prop/direct/>
            SELECT  ?label
            WHERE{
              ?item wdt:P31/wdt:P279* wd:Q11424 .
              ?item wdt:P136 wd:'''+genre+''' .
              ?item rdfs:label ?label .
              FILTER(LANG(?label) = "en").
            } LIMIT 1000
                '''
        response = moviegraph.query(query)
        labels = []
        for e in response:
            labels.append(str(e[0]))
        df = pd.DataFrame(response.bindings, columns = ["director", "subject", "period"])
        df['label'] = labels
        return df

In [10]:
class Recommander():
    def __init__(self, moviegraph, queryproducer):
        self.queryproducer = queryproducer
        self.moviegraph = moviegraph

    def decompose_question(self, question):
        try:
            if "," in question:
                sub_sentences = question.split(",")

            else:
                sub_sentences = question.split("and")
        except:
            return False
        return sub_sentences

    def find_simularities(self, movies):
        genre_id = self.queryproducer.predicateID_query("genre")
        director_id = self.queryproducer.predicateID_query("director")
        main_subject_id = self.queryproducer.predicateID_query("main subject")
        set_in_period_id = self.queryproducer.predicateID_query("set in period")

        try:
            genres = [self.queryproducer.any_property_value(movie, genre_id, "genre") for movie in movies]
            directors = [self.queryproducer.any_property_value(movie, director_id, "director") for movie in movies]
            main_subjects = [self.queryproducer.any_property_value(movie, main_subject_id, "main subject") for movie in movies]
            set_in_periods = [self.queryproducer.any_property_value(movie, set_in_period_id, "set in period") for movie in movies]
        except:
            print("could not find properties of given movies")
            return False

        properties = [genres, directors, main_subjects, set_in_periods]
        for e in properties:
            if "nothing" in e:
                e.remove("nothing")

####### Collect the most common properties
        genre = mode(genres)
        director = mode(directors)
        main_subject = mode(main_subjects)
        set_in_period = mode(set_in_periods)
        print("common genre: " + str(genre))

        try:
            genre_value_id = self.queryproducer.genre_id(genre)
        except:
            print("could not find the id of the genre of recommander movies")
            return False
##### Get all movies with the given genre, and some other properties
        if genre:
            try:
                df = self.queryproducer.all_genre_films(genre_value_id)
                print("found movies of same genre: ")
                print(df.shape)
            except:
                print("could not find movie sof same genre")
                return False

            for movie in movies:
                try:
                    df.drop(df[df["label"] == movie].index, inplace = True)
                except:
                    print("could not delete instances of same movies")

            for label in df["label"].values.tolist():
                try:
                    director_value = Q.any_property_value(label, director_id, "director")
                    subject_value = Q.any_property_value(label, main_subject_id, "main subject")
                    period_value = Q.any_property_value(label, set_in_period_id, "set in period")
                    df.loc[df.label == label, "director"] = str(director_value)
                    df.loc[df.label == label, "subject"] = str(subject_value)

                    if period_value.endswith("s"):
                        period_value = period_value[:-1]
                    df.loc[df.label == label, "period"] = str(period_value)
                except:
                    print("Could not add properties to dataframe")
                    pass

        else:
            return False

###### try to find a movies that share common properties
        print("found common genre")
        common = {"genre": genre }

        if not main_subject == "nothing":
            df_subject = df.loc[df.subject == main_subject]
            if not df_subject.empty:
                print("found common subjects")
                common["main subject"] = main_subject
                df = df_subject

        if not set_in_period == "nothing":
            df_period = df.loc[df.period == set_in_period]
            if not df_period.empty:
                print("found common set in period")
                common["set in period"] = set_in_period
                df = df_period

        if not director == "nothing":
            df_director = df.loc[df.director == director]
            if not df_director.empty:
                print("found common director")
                common["director"] = director
                df = df_director

        film_titles = df["label"].tolist()
        if len(film_titles) > 3:
            film_titles = film_titles[:3]
        common["film_titles"]= film_titles

        return common

In [11]:
KGloader = KGLoader()
moviegraph = KGloader.load_KG()
embedding = KGloader.load_embeddings(moviegraph)
print("loaded KG and embeddings")


Loading knowledge graph from source
succesfully loaded knowledge graph
loaded KG and embeddings


In [12]:
Q = QueryProducer(moviegraph)
actors_ID_dict = Q.all_actors_id_and_IMDb_query()[0]
actor_IMDb_dict = Q.all_actors_id_and_IMDb_query()[1]
all_films_dict = Q.all_films_id_query()
print("loaded film and actor dicts")

found normal films; dict has size 25509
found subcategories; dict has size 2500
len of subcategories: 59
found sub_subcategories; dict has size 0
found animes; dict has size 114
None
the final film dict has size 25509
loaded film and actor dicts


In [13]:
image_frame = pd.DataFrame.from_dict(json.load(open("images.json")))
print("loaded image file")

loaded image file


In [44]:
class LookupHandler:
    def __init__(self, film_ids, actor_ids, actor_IMDb, query_producer, moviegraph, embedding, images, crowdsourcing):
        self.film_ids = film_ids

        print("titanic" in self.film_ids.keys())

        self.actor_IDs = actor_ids
        self.actor_IMDb = actor_IMDb
        self.query_producer = query_producer
        self.movie_graph = moviegraph
        self.embedding = embedding
        self.images = images
        self.crowdsourcing = crowdsourcing
    
    def film_id_finder(self, film_title):
        # Film entity. Using lowercase title for lookup
        try:
            film_id = self.film_ids[film_title.lower()]
            print("found film ID " + str(film_id))
            return film_id
        except:
            print("could not find film, trying to find it correcting typos...")
            pass
        try:
            film_title = TypoCorrector.remove_last_digit_space(film_title)
            film_id = self.film_ids[film_title.lower()]
            print("found film ID " + str(film_id))
            return film_id
        except:
            print("could not find film, trying to find it correcting typos...")
            pass
        try:
            film_title = TypoCorrector.change_minus_sign(film_title)
            film_id = self.film_ids[film_title.lower()]
            print("found film ID " + str(film_id))
            return film_id
        except:
            pass
        try:
            film_title = TypoCorrector.change_dash_sign(film_title)
            film_id = self.film_ids[film_title.lower()]
            print("found film ID " + str(film_id))
            return film_id
        except:
            print("could not find ID in id_dict of film: "+ film_title)
            raise KeyError("Film title contains typos or does not exist!")

    def graph_lookup(self, film_title, pred, pred_id):
        
        query = self.query_producer.any_property_value_query(title=film_title, id=pred_id, propertyname=pred)
        entity = self.movie_graph.query(query)
        df_entity = pd.DataFrame(entity.bindings)

        if df_entity.empty:
            film_title = TypoCorrector.change_minus_sign(film_title)
            query = self.query_producer.any_property_value_query(title=film_title, id=pred_id, propertyname=pred)
            entity = self.movie_graph.query(query)
            df_entity = pd.DataFrame(entity.bindings)

            if df_entity.empty:
                film_title = TypoCorrector.change_dash_sign(film_title)
                query = self.query_producer.any_property_value_query(title=film_title, id=pred_id, propertyname=pred)
                entity = self.movie_graph.query(query)
                df_entity = pd.DataFrame(entity.bindings)

                if df_entity.empty:
                    film_title = TypoCorrector.remove_last_digit_space(film_title)
                    query = self.query_producer.any_property_value_query(title=film_title, id=pred_id, propertyname=pred)
                    entity = self.movie_graph.query(query)
                    df_entity = pd.DataFrame(entity.bindings)

        # takes all answers but responds with the first one
        property_value = []
        for _ in entity:
            property_value.append(str(_.name))
        return property_value

    def embedding_lookup(self, film_title, predicate, pred_id):
        entity_emb, relation_emb, ent2id, id2ent, rel2id, id2rel, ent2lbl, lbl2ent = self.embedding

        # define some prefixes
        WD = rdflib.Namespace('http://www.wikidata.org/entity/')
        WDT = rdflib.Namespace('http://www.wikidata.org/prop/direct/')

        film_id = self.film_id_finder(film_title)
        head = entity_emb[ent2id[WD[film_id]]]

        pred = relation_emb[rel2id[WDT[pred_id]]]


        # add vectors according to TransE scoring function.
        lhs = head + pred
        # compute distance to *any* entity
        dist = pairwise_distances(lhs.reshape(1, -1), entity_emb).reshape(-1)
        # find most plausible entities
        most_likely = dist.argsort()
        # compute ranks of entities
        ranks = dist.argsort().argsort()

        top_ten_results = pd.DataFrame([
                (id2ent[idx][len(WD):], ent2lbl[id2ent[idx]], dist[idx], rank+1)
                for rank, idx in enumerate(most_likely[:10])],
                columns=('Entity', 'Label', 'Score', 'Rank'))
        top_three = top_ten_results.iloc[0:3]["Label"]
        top_three.values.tolist()

        return top_three

    def image_lookup(self, name, asFilmIdentifiedName):
        try:
            if name.endswith(" "):
                name = name[:-1]
        except:
            pass

        try:
            if asFilmIdentifiedName.endswith(" "):
                name = asFilmIdentifiedName[:-1]
        except:
            pass

        try:
            actor_Id = self.actor_IDs[name]
        except:
            actor_Id = self.actor_IDs[asFilmIdentifiedName]

        IMDb_Id = self.actor_IMDb[actor_Id]
        valid_options = self.images.loc[self.images["cast"].str.contains(IMDb_Id, regex=False)]
    
        #only images where actor appears alone and its a poster. Take a random sample
        try:
            valid_options = valid_options.loc[valid_options["type"]=="poster"]
        except:
            pass
        try:
            valid_options = valid_options.query("cast.str.len() == 1", engine="python")
        except:
            pass
        valid_options = valid_options["img"]
        valid_options = valid_options.sample()

        url = str(valid_options.iloc[0])
        return "image:"+url[:-4]

    # Receives and returns a dict with batch IDm batch kappa, the verdict,  agrrement nr, disagreement nr, and the value
    def crowdsourcing_lookup(self, filmtitle, pred_id):
        film_id = self.film_id_finder(filmtitle)
        answer = self.crowdsourcing.crowdsourcing_answer(film_id, pred_id)
        print("Value is: " + answer["value"])
        if answer["value"].startswith("wd:"):
            label = self.query_producer.label_query(answer["value"])
            answer["value"] =  label
            return answer
        if answer["value"].startswith("Q"):
            label = self.query_producer.label_query("wd:"+answer["value"])
            answer["value"] =  label
            return answer
        return answer

In [66]:
#Takes an argument "training" to determine if it should create and save a model or load an existing one
class RunProcess:
    def __init__(self, training, knowledgegraph):
        print("Starting process")
        self.ner_model = NerTrainer()
        if training:
            self.crf_model = self.ner_model.train()
            print("Created and trained model succesfully")
            with open('crf_model.pkl', 'wb') as f:
                pickle.dump(self.crf_model, f)
            print("Saved crf-model")
        else:
            #self.ner_model = NerDLModel.pretrained('ner_mit_movie_simple_distilbert_base_cased', 'en')\
            #.setInputCols(['document', 'token', 'embeddings']).setOutputCol('ner')
            with open('crf_model.pkl', 'rb') as f:
                self.crf_model = pickle.load(f)
            print("loaded ner-mnodel")

        self.movie_graph = knowledgegraph
        self.embedding = embedding
        self.actor_IDs = actors_ID_dict
        self.actor_IMDb = actor_IMDb_dict
        self.film_ids = all_films_dict #combine both to one dict
        self.images = image_frame

        self.crowdourcing = Crowdsourcing()
        self.question_analyser = QuestionAnalyser()
        self.query_producer = QueryProducer(self.movie_graph)
        self.lookup_handler = LookupHandler(self.film_ids, self.actor_IDs, self.actor_IMDb,
            self.query_producer, self.movie_graph, self.embedding, self.images, self.crowdourcing)
        self.recommander = Recommander(self.movie_graph, self.query_producer)

        print("----------Run Initialized!----------")

    def answer_movie_question(self, *args):
        if not args:
            question = self.ask_question()
        else:
            print(args)
            question = args[0]["message"]
        film_title, actor, pred = self.identify_subject_and_predicate(question)
        print("predicate found and handed to answer_movie_question: ")
        print(pred)

        if pred == "recommendation":
            if question[-1] in [".", "?", "!"]:
                question = question[:-1]
            movies = self.recommander.decompose_question(question= question)
            if not movies:
                answer = ["Fail1"]
                answer = self.answer_designer(film_title="", answer=answer, pred="recommendation")
                return answer
            else:
                subjects = []

                for movie in movies:
                    tagged_entities = self.ner_model.predict_tags(self.crf_model, movie)
                    film_wordlist = tagged_entities['WORD'][tagged_entities['TAG'].isin(['B-TITLE','I-TITLE'])]
                    film_title = ''
                    for w in film_wordlist:
                        film_title = film_title + w + ' '
                    if film_title.endswith(" "):
                        film_title = film_title[:-1]
                    subjects.append(film_title)

                answer = self.recommander.find_simularities(subjects)
                if not answer:
                    answer = ["Fail2"]
                    answer = self.answer_designer(film_title="", answer=answer, pred="recommendation")
                else:
                    answer = self.answer_designer(film_title="", answer=answer, pred="recommendation")
            return answer

        #set pred to defualt value
        if not pred:
            pred = "director"
        # some predicates cannot be fetched by the query. Their
        if pred in self.question_analyser.exceptional_predicates.keys():
            pred_id = self.question_analyser.handle_exceptional_predicates(pred)
        elif pred[:-1] in self.question_analyser.exceptional_predicates.keys():
            pred_id = self.question_analyser.handle_exceptional_predicates(pred[:-1])
        else:
            pred_id = self.query_producer.predicateID_query(pred)

        film_title, actor = TypoCorrector.title_corrector(film_title, actor)

        print("subject: "+ str(film_title) + " | actor: " +str(actor)+ " | predicate: "+str(pred)+" | predicate ID: "+str(pred_id) )
        print("##################################################################")

        answer_set = self.lookup(actor=actor, film_title=film_title, pred=pred, pred_id=pred_id)

        answer = self.answer_designer(film_title=film_title, pred=pred, answer=answer_set)

        return answer

    def ask_question(self):
        question = input("Please ask your question")
        return question

    def identify_subject_and_predicate(self, question):
        tagged_entities = self.ner_model.predict_tags(self.crf_model, question)
        print("Sentence with tags: ")
        print(tagged_entities)
        print("#################################################################")
        identified_predicate = self.question_analyser.identify_predicate(tagged_entities)

        if not identified_predicate:
            recommender = ["Recommend", "recommend", "Recommendation", "recommendation"]
            if any(c in tagged_entities["WORD"].unique() for c in recommender):
                return 0, 0, "recommendation"


        if "B-TITLE" not in tagged_entities["TAG"].unique():
            print("changing Tags because no title tags found")
            try:
                idx = tagged_entities.index[tagged_entities["TAG"]=="I-PLOT"].tolist()
                tagged_entities.iloc[idx[0]:idx[0]+1]["TAG"] = 'B-TITLE'
                tagged_entities.loc[tagged_entities['TAG'] == "I-PLOT", 'TAG'] = 'I-TITLE'
            except:
                print("could not add Title Tags")


        #TODO Here we can also check for actors. ALSO we can check if I-title is a questions mark, comma etc.
        film_wordlist = tagged_entities['WORD'][tagged_entities['TAG'].isin(['B-TITLE','I-TITLE'])]
        film_title = ''
        for w in film_wordlist:
            film_title = film_title + w + ' '

        actor_wordlist = tagged_entities['WORD'][tagged_entities['TAG'].isin(['B-ACTOR','I-ACTOR'])]
        actor = ''
        for w in actor_wordlist:
            actor = actor + w + ' '
        # need to remove empry string at end of concatenation
        film_title = film_title[:-1]
        actor = actor[:-1]

        return film_title, actor, identified_predicate


        film_id = self.film_id_finder(filmtitle)
        answer = self.crowdourcing.crowdsourcing_answer(film_id, pred_id)
        return answer

    def lookup(self, actor, film_title, pred, pred_id):
        answer = {}
        print("doing lookup for: " + str(actor) +" |"+ str(film_title) + " |" + str(pred))

        if pred == 'image':
            try:
                image_url = self.lookup_handler.image_lookup(name = actor, asFilmIdentifiedName=film_title)
                print("Image of " + film_title + actor + " at this url: "+ image_url)
                answer["image"] = image_url
            except:
                answer["image"] = "Unfortunately I was unable to find an image of:" + film_title + actor
                pass

        else:
            try:
                answer_crowdsourcing = self.lookup_handler.crowdsourcing_lookup(film_title, pred_id)
                answer["crowdsourcing"] = answer_crowdsourcing
                print("found crowdsourcing answer")
            except:
                print("Could not find crowd answer")
                pass

            try:
                # this answer might have multiple answers in a list
                answer_graph = self.lookup_handler.graph_lookup(film_title, pred, pred_id)
                answer["graph"] = answer_graph[0]
                print("found KG answer")
            except:
                print("Could not find graph answer")
                pass

            try:
                answer_embedding = self.lookup_handler.embedding_lookup(film_title, pred, pred_id)
                answer["embedding"] = answer_embedding
                print("found embedding answer")
            except:
                print("Could not embedding crowd answer")
                pass
        return answer

    def answer_designer(self, film_title, pred, answer):
        answer_sentence = ""

        if pred == "recommendation":
            if answer == "Fail1":
                return "I'm sorry, but I could not properly identify the movies you gave me, hence I cant give you any recommendation."
            if answer == "Fail2":
                return "I'm sorry, but I could not find any proper recommendation that matches the movies you gave me!"
            else:
                answer_sentence = "According to the movies you gave me, I was looking for movies of the genre: " +answer["genre"]
                if "main subject" in answer.keys():
                    answer_sentence + ", the mains subject " + answer["main subject"]
                if "set in period" in answer.keys():
                    answer_sentence + ", where the set in period is " + answer["set in period"]
                if "director" in answer.keys():
                    answer_sentence + " and the director is " + answer["director"] + "."
                if len(answer["film_titles"]) == 0:
                    answer_sentence + "Unfortunately I was not able to find any movies matching at least one criteria."
                else:
                    films = ". Those are the film that I can recommend you: "
                    for film in answer["film_titles"]:
                        films = films + film + ", "
                    answer_sentence = answer_sentence + films[:-1] +"."
                return answer_sentence


        if not answer:
            answer_sentence = "I'm sorry but I could not find an answer to your question. Double check for typos or try to ask a different question"
        else:
            # answer is a dict with keys: image, crowdsourcing, graph, embedding
            keys = answer.keys()
            # if we need to return an image, its easy
            if "image" in keys:
                return answer["image"]

            # Then we check for the other possible answers
            # Keep in mind that the answers might consist of iterals
            # We will setup the answer as a combination of multiple

            # First we check if the knowledgegraph gave us something and if yes, if the embedding can React to it.
            if "graph" in keys:
                answer_sentence = answer_sentence + "According to the knowledge-graph, the " + str(pred) + " of " + str(film_title) + " is " + str(answer["graph"]) + "."
                if "embedding" in keys:
                    if answer["graph"] == answer["embedding"][0]:
                        print("graph - embeddding case 1")
                        answer_sentence = answer_sentence + " The top answer by the embedding is the same, while having " + str(answer["embedding"][1]) + " as second most likely answer."
                    elif answer["graph"] == answer["embedding"][1]:
                        print("graph - embeddding case 2")
                        answer_sentence = answer_sentence + " The embedding actually suggests " + str(answer["embedding"][0]) +" as most likely answer, while " + str(answer["embedding"][1]) + " only comes second."
                    elif answer["graph"] not in  answer["embedding"]:
                        print("graph - embeddding case 3")
                        answer_sentence = answer_sentence + " The embedding actually suggests either " + str(answer["embedding"][0]) +" or " + str(answer["embedding"][1]) + " as more likely answers though."

            # Since there was no answer by the knowledge-graph, we have to rephrase the sentences
            else:
                if "embedding" in keys:
                    answer_sentence = answer_sentence +  "The embedding suggests that " + str(answer["embedding"][0]) +" is the most likely answer and " + str(answer["embedding"][1]) + " as the second most likely answers."

            if "crowdsourcing" in keys:
                batch_id = answer["crowdsourcing"]["batch"]
                kappa = str(answer["crowdsourcing"]["kappa"])
                verdict = answer["crowdsourcing"]["verdict"]
                agreement = answer["crowdsourcing"]["agreement"]
                disagreement = answer["crowdsourcing"]["disagreement"]
                value = str(answer["crowdsourcing"]["value"])

                if verdict =="True":
                    answer_sentence = answer_sentence + " When concerning crowdsourcing, the crowd agrees that " + value + " is the " + str(pred) + " of " + str(film_title)[:-1]+ ". " + str(agreement) +" vote in favour of this statement and "+ str(disagreement) + " vote against it. The interrater agreement of this batch is "+kappa+"."

                elif verdict =="False":
                    answer_sentence = answer_sentence + " When concerning crowdsourcing, the crowd diagrees that " + value + " is the " + str(pred) + " of " + str(film_title)[:-1]+ ". "+ str(agreement) +" vote in favour of this statement and "+ str(disagreement) + " vote against it. The interrater agreement of this batch is "+kappa+"."

                elif verdict =="corrected":
                    answer_sentence = answer_sentence + " When concerning crowdsourcing, the crowd disagreed on the proposed statement, but suggests that that " + value + " is the actual " + str(pred) + " of " + str(film_title)+ ". " +"The interrater agreement of this batch is "+kappa+"."

                elif verdict=="tie":
                    answer_sentence = answer_sentence + " When concerning crowdsourcing, the crowd is undecided if " + value + " is the " + str(pred) + " of " + str(film_title)[:-1]+ ". "+ str(agreement) +" vote in favour of this statement and "+ str(disagreement) + " vote against it. The interrater agreement of this batch is "+kappa +"."

        return answer_sentence

In [67]:
# url of the speakeasy server
url = 'https://speakeasy.ifi.uzh.ch'
listen_freq = 3


class DemoBot:
    def __init__(self, username, password):
        self.agent_details = self.login(username, password)
        self.session_token = self.agent_details['sessionToken']
        self.chat_state = defaultdict(lambda: {'messages': defaultdict(dict), 'initiated': False, 'my_alias': None})
        self.process = RunProcess(training=False, knowledgegraph=moviegraph)

        atexit.register(self.logout)

    def listen(self):
        while True:
            # check for all chatrooms
            current_rooms = self.check_rooms(session_token=self.session_token)['rooms']
            for room in current_rooms:
                # ignore finished conversations
                if room['remainingTime'] > 0:
                    room_id = room['uid']
                    if not self.chat_state[room_id]['initiated']:
                        # send a welcome message and get the alias of the agent in the chatroom
                        self.post_message(room_id=room_id, session_token=self.session_token, message='Hi, you can send me any message and check if it is echoed in {} seconds.'.format(listen_freq))
                        self.chat_state[room_id]['initiated'] = True
                        self.chat_state[room_id]['my_alias'] = room['alias']

                    # check for all messages
                    all_messages = self.check_room_state(room_id=room_id, since=0, session_token=self.session_token)['messages']

                    # you can also use ["reactions"] to get the reactions of the messages: STAR, THUMBS_UP, THUMBS_DOWN

                    for message in all_messages:
                        if message['authorAlias'] != self.chat_state[room_id]['my_alias']:

                            # check if the message is new
                            if message['ordinal'] not in self.chat_state[room_id]['messages']:
                                self.chat_state[room_id]['messages'][message['ordinal']] = message
                                print('\t- Chatroom {} - new message #{}: \'{}\' - {}'.format(room_id, message['ordinal'], message['message'], self.get_time()))

                                try:
                                    answer = self.process.answer_movie_question(message)
                                except:
                                    answer = "Sorry I coult not answer your question, please try something else."

                                #self.post_message(room_id=room_id, session_token=self.session_token, message='Got your message: \'{}\' at {}.'.format(message['message'], self.get_time()))
                                self.post_message(room_id=room_id, session_token=self.session_token, message=answer)
            time.sleep(listen_freq)

    def login(self, username: str, password: str):
        agent_details = requests.post(url=url + "/api/login", json={"username": username, "password": password}).json()
        print('- User {} successfully logged in with session \'{}\'!'.format(agent_details['userDetails']['username'], agent_details['sessionToken']))
        return agent_details

    def check_rooms(self, session_token: str):
        return requests.get(url=url + "/api/rooms", params={"session": session_token}).json()

    def check_room_state(self, room_id: str, since: int, session_token: str):
        return requests.get(url=url + "/api/room/{}/{}".format(room_id, since), params={"roomId": room_id, "since": since, "session": session_token}).json()

    def post_message(self, room_id: str, session_token: str, message: str):
        tmp_des = requests.post(url=url + "/api/room/{}".format(room_id),
                                params={"roomId": room_id, "session": session_token}, data=message).json()
        if tmp_des['description'] != 'Message received':
            print('\t\t Error: failed to post message: {}'.format(message))

    def get_time(self):
        return time.strftime("%H:%M:%S, %d-%m-%Y", time.localtime())

    def logout(self):
        if requests.get(url=url + "/api/logout", params={"session": self.session_token}).json()['description'] == 'Logged out':
            print('- Session \'{}\' successfully logged out!'.format(self.session_token))


In [68]:
username = 'dean.heizmann_bot'
#password = getpass.getpass('Password of the demo bot:')
demobot = DemoBot(username, "6dNtSBEkgDAl0g")
demobot.listen()

	- Chatroom 3c24343c-065a-498d-b8af-fe931b3ab37d - new message #33: 'What was the filming location of Titanic?' - 21:37:54, 12-12-2022
({'timeStamp': 1670877472170, 'authorAlias': 'awesome_greider_2', 'ordinal': 33, 'message': 'What was the filming location of Titanic?'},)
Sentence with tags: 
       WORD  POS      TAG
0      What   WP        0
1       was  VBD        0
2       the   DT        0
3   filming   JJ        0
4  location   NN        0
5        of   IN        0
6   Titanic  NNP  B-TITLE
7         ?    .  I-TITLE
#################################################################
['filming', 'location']
['wa']
max valu in identity counter: 
filming location 1
pred to return:
filming location
predicate found and handed to answer_movie_question: 
filming location
subject: Titanic  | actor:  | predicate: filming location | predicate ID: P915
##################################################################
doing lookup for:  |Titanic  |filming location
could not find film, trying

KeyboardInterrupt: 

In [ ]:
Process3 = RunProcess(training=False, knowledgegraph=moviegraph)

In [ ]:
print(Process3.answer_movie_question())

In [106]:
'''
Tell me the publication date of Spy Game
Recommend movies like Nightmare on Elm Street, Friday the 13th, and Halloween.
Recommend movies like X-Men: First Class, The Notebook, and Titanic.
"Titanic", "The Notebook", "X-Men: First Class?"
Who is the director of Good Will Hunting?

Who produced The Lion King?
Tell me the publication date of The Lion King.

What is the Kinopoisk film ID of The Notebook?
Tell me the exploitation visa number of X-Men: First Class?

Who is the screenwriter of The Masked Gang: Cyprus
Who is the screenwriter of Titanic?
Who is the screenwriter of Weathering with You

Who is the director of Star Wars: Episode VI - Return of the Jedi?
Who is the executive producer of X-Men: First Class?
What is the genre of The Lion King?
What is the MPA film rating of The Lion King?
What is the MPA film rating of The Notebook?

What is the MPAA film rating of Weathering with You?
What is the MPAA film rating of Titanic?
What is the box office of The Princess and the Frog?
Can you tell me the publication date of Tom Meets Zizou?

Show me a picture of Julia Roberts.

The Lion King
The Notebook
Titanic
'''

'\nRecommend movies like Nightmare on Elm Street, Friday the 13th, and Halloween.\nRecommend movies like X-Men: First Class, The Notebook, and Titanic.\n"Titanic", "The Notebook", "X-Men: First Class?"\nWho is the director of Good Will Hunting?\n\nWho produced The Lion King?\nTell me the publication date of The Lion King.\n\nWhat is the Kinopoisk film ID of The Notebook?\nTell me the exploitation visa number of X-Men: First Class?\n\nWho is the screenwriter of The Masked Gang: Cyprus\nWho is the screenwriter of Titanic?\nWho is the screenwriter of Weathering with You\n\nWho is the director of Star Wars: Episode VI - Return of the Jedi?\nWho is the executive producer of X-Men: First Class?\nWhat is the genre of The Lion King?\nWhat is the MPA film rating of The Lion King?\nWhat is the MPA film rating of The Notebook?\n\nWhat is the MPAA film rating of Weathering with You?\nWhat is the MPAA film rating of Titanic?\nWhat is the box office of The Princess and the Frog?\nCan you tell me the

In [ ]:
"Tell me the publication data of Spy Game"

In [55]:
Q = QueryProducer(knowledgegraph=moviegraph)

value = Q.any_property_value("Spy Game", "P2047", "duration")

print(value)

action film


In [61]:
query = ''' prefix wd: <http://www.wikidata.org/entity/>
        prefix wdt: <http://www.wikidata.org/prop/direct/>

        SELECT ?value
        WHERE {
            wd:Q1049139 wdt:P136 ?value .
        }
'''

entity = moviegraph.query(query)
print(entity.bindings)

[{rdflib.term.Variable('value'): rdflib.term.URIRef('http://www.wikidata.org/entity/Q188473')}, {rdflib.term.Variable('value'): rdflib.term.URIRef('http://www.wikidata.org/entity/Q21401869')}, {rdflib.term.Variable('value'): rdflib.term.URIRef('http://www.wikidata.org/entity/Q2297927')}, {rdflib.term.Variable('value'): rdflib.term.URIRef('http://www.wikidata.org/entity/Q3990883')}, {rdflib.term.Variable('value'): rdflib.term.URIRef('http://www.wikidata.org/entity/Q622291')}]
